*Note: this notebook was generated by dsflow*

# create table meteo_agg

## \[1- init and config\] refresh tables and set output path

In [1]:
import os
import json
import datetime as dt
from pandasql import PandaSQL
from custom_libraries.helpers import show_tables, load_table

In [22]:
show_tables()

camera_agg
car_speed_limits
car_speed_limits_agg
dim_users
meteo_agg
meteoparis
users_agg


In [17]:
default_task_specs = """
    {"sink_path": "/data/tables/meteo_agg/ds=2018-01-01",
     "ds": "2017-11-24"}
    """

In [18]:
task_specs_raw = os.environ.get('TASK_SPECS', default_task_specs)
task_specs = json.loads(task_specs_raw)

print(task_specs)

{'sink_path': '/data/tables/meteo_agg/ds=2018-01-01', 'ds': '2017-11-24'}


## \[2- process data\] process data based on SQL

In [5]:
meteoparis = load_table(type="parquet", table_name="meteoparis")

Ignoring path: /data/tables/meteoparis/ds=2017-12-29/_SUCCESS
Ignoring path: /data/tables/meteoparis/ds=2017-12-30/_SUCCESS
Ignoring path: /data/tables/meteoparis/ds=2017-12-31/_SUCCESS
Ignoring path: /data/tables/meteoparis/ds=2018-01-01/_SUCCESS


In [11]:
psql = PandaSQL()
df = psql("""

  SELECT cast(forecast as timestamp) as forecast_hour,
     sum(total_water_precipitation) as total_precipitation
  FROM meteoparis
  WHERE ds = '2018-01-01'
  GROUP BY 1
  ORDER BY 1

""")

Execute query and show first 10 rows

In [12]:
df

,forecast_hour,total_precipitation
0,2018,39670.931091


In [15]:
# print schema
df.columns

Index(['forecast_hour', 'total_precipitation'], dtype='object')

## \[3- write on disk\] write the output as parquet

In [19]:
final_dataset = df

In [21]:
base_dir = "/data/tables/meteo_agg/ds=2018-01-01"

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

final_dataset.to_parquet(task_specs["sink_path"] + "/data.parquet")